In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.auto import tqdm

In [2]:
import numpy as np


def prune_contributions(contributions):
    """
    Prune contributions by summing values for each unique coordinate and removing
    any contributions that result in a total value of zero.

    Parameters
    ----------
    contributions : list of tuples
        A list where each element is a tuple containing a coordinate (tuple/list) and a contribution value.
        Coordinates may repeat, and the function will sum the contributions with the same coordinates.

    Returns
    -------
    list of tuples
        A sorted list of tuples, where each tuple contains a unique coordinate and its total contribution,
        excluding coordinates with a zero contribution.
    """
    total_ECP = dict()

    # Sum contributions for each unique coordinate
    for a in contributions:
        total_ECP[a[0]] = total_ECP.get(a[0], 0) + a[1]

    # Remove the contributions that are zero
    to_del = [key for key, value in total_ECP.items() if value == 0]
    for key in to_del:
        del total_ECP[key]

    # Return sorted list of tuples (coordinate, total contribution)
    return sorted(total_ECP.items(), key=lambda x: x[0])


def EC_at_value(contributions, *fs):
    """
    Calculate the sum of contributions for a given set of coordinates in n-dimensional space.

    Parameters
    ----------
    contributions : list of tuples
        A list where each element is a tuple containing a coordinate and a contribution value.

    *fs : float
        A variable number of threshold values representing the coordinates for each dimension.
        The number of values must match the number of dimensions of the coordinates in the contributions.

    Returns
    -------
    float
        The sum of contributions that are less than or equal to the given thresholds for each dimension.
    """
    return sum(
        [c[1] for c in contributions if all(c[0][i] <= fs[i] for i in range(len(fs)))]
    )


def difference_ECP(ecp_1, ecp_2, dims):
    """
    Calculate the difference in contributions between two ECPs (Energy Contribution Points)
    for an arbitrary number of dimensions.

    Parameters
    ----------
    ecp_1 : list of tuples
        A list of tuples, where each tuple contains a coordinate (in any dimension) and a contribution value.

    ecp_2 : list of tuples
        A list of tuples, where each tuple contains a coordinate (in any dimension) and a contribution value.

    dims : tuple of float
        A tuple representing the min and max boundaries for each dimension. The length of `dims` must be twice the number
        of dimensions, i.e., for `n` dimensions, `dims` should contain `2n` values.

    Returns
    -------
    float
        The difference in the energy contributions, calculated by subtracting contributions from `ecp_2`
        from `ecp_1`, integrating over the space defined by `dims`.

    Notes
    -----
    The function assumes that the dimensions in the `dims` parameter are ordered in the form
    `(f0min, f0max, f1min, f1max, ..., fnmin, fnmax)`, where each pair of consecutive values corresponds
    to the min and max of a specific dimension. The contributions should have coordinates matching the number of
    dimensions in `dims`.

    Examples
    --------
    ecp_1 = [((1, 2, 3), 10), ((4, 5, 6), 20)]
    ecp_2 = [((2, 3, 4), 5), ((3, 4, 5), 15)]
    dims = (0, 5, 0, 5, 0, 5)  # 3D space, with x, y, z ranging from 0 to 5
    result = difference_nd_ECP(ecp_1, ecp_2, dims)
    print(result)
    """

    # Extract the min and max values for each dimension from `dims`
    num_dims = len(dims) // 2
    mins = dims[::2]
    maxs = dims[1::2]

    # Initialize contributions with the boundary points
    contributions = [
        (tuple([mins[i] for i in range(num_dims)]), 0),
        (tuple([maxs[i] for i in range(num_dims)]), 0),
    ]

    contributions += ecp_1
    contributions += [(c[0], -1 * c[1]) for c in ecp_2]

    # Prune contributions and add back boundary points
    contributions = (
        [(tuple([mins[i] for i in range(num_dims)]), 0)]
        + prune_contributions(contributions)
        + [(tuple([maxs[i] for i in range(num_dims)]), 0)]
    )

    # Generate sorted lists of unique coordinates for each dimension
    coordinate_lists = [
        sorted(set([c[0][i] for c in contributions])) for i in range(num_dims)
    ]

    # Initialize the difference variable
    difference = 0

    # Initialize indices for each dimension (we start from the first coordinate)
    indices = [0] * num_dims

    # List to store deltas for each dimension (initialized to None, we'll calculate them later)
    deltas = [None] * num_dims

    while True:
        # Calculate the deltas for the current indices, i.e., the differences between adjacent coordinates
        for i in range(num_dims):
            deltas[i] = (
                coordinate_lists[i][indices[i] + 1] - coordinate_lists[i][indices[i]]
            )

        # Calculate the coordinates corresponding to the current indices
        coords = tuple(coordinate_lists[i][indices[i]] for i in range(num_dims))

        # Compute the contribution at the current coordinates
        contribution_value = EC_at_value(contributions, *coords)

        # Multiply the contribution by the volume of the current "box" (product of deltas)
        difference += contribution_value * np.prod(deltas)

        # Move to the next set of indices (advance the indices like counting in a multi-dimensional array)
        for i in range(
            num_dims - 1, -1, -1
        ):  # Start from the last dimension and move backwards
            indices[i] += 1
            if indices[i] < len(coordinate_lists[i]) - 1:
                break  # If we've not overflowed, stop incrementing
            else:
                indices[i] = 0  # Reset this dimension and move to the next dimension

        # If we've exhausted all indices, break out of the loop
        if all(idx == 0 for idx in indices):
            break

    return difference

In [3]:
rng = np.random.default_rng(seed=42)

## 2d

create two ECP and compute their distance

In [13]:
contributions_2d = [
    (tuple(rng.integers(low=0, high=99, size=2)), rng.integers(low=-5, high=5))
    for _ in range(10)
]

pd.DataFrame(data=[[row[0][0], row[0][1], row[1]] for row in contributions_2d]).to_csv(
    "data/distances/2d_1.csv", index=False, header=False, sep=" "
)

contributions_2d

[((84, 27), 4),
 ((29, 42), 1),
 ((12, 55), 0),
 ((77, 98), 1),
 ((40, 40), -1),
 ((80, 31), -4),
 ((33, 2), -4),
 ((8, 76), 2),
 ((68, 45), 2),
 ((15, 89), 0)]

In [14]:
new_contributions_2d = contributions_2d + [((50, 50), 1)]

pd.DataFrame(
    data=[[row[0][0], row[0][1], row[1]] for row in new_contributions_2d]
).to_csv("data/distances/2d_2.csv", index=False, header=False, sep=" ")

new_contributions_2d

[((84, 27), 4),
 ((29, 42), 1),
 ((12, 55), 0),
 ((77, 98), 1),
 ((40, 40), -1),
 ((80, 31), -4),
 ((33, 2), -4),
 ((8, 76), 2),
 ((68, 45), 2),
 ((15, 89), 0),
 ((50, 50), 1)]

In [6]:
assert (
    difference_ECP(contributions_2d, new_contributions_2d, dims=(0, 100, 0, 100))
    == -50 * 50
)

## 3d

In [7]:
contributions_3d = [
    (tuple(rng.integers(low=0, high=99, size=3)), rng.integers(low=-5, high=5))
    for _ in range(10)
]

pd.DataFrame(
    data=[[row[0][0], row[0][1], row[0][2], row[1]] for row in contributions_3d]
).to_csv("data/distances/3d_1.csv", index=False, header=False, sep=" ")

new_contributions_3d = contributions_3d + [((50, 50, 50), -1)]

pd.DataFrame(
    data=[[row[0][0], row[0][1], row[0][1], row[1]] for row in new_contributions_3d]
).to_csv("data/distances/3d_2.csv", index=False, header=False, sep=" ")

In [8]:
assert (
    difference_ECP(contributions_3d, new_contributions_3d, (0, 100, 0, 100, 0, 100))
    == 50**3
)

## 4d

In [ ]:
contributions_4d = [
    (tuple(rng.integers(low=0, high=99, size=4)), rng.integers(low=-5, high=5))
    for _ in range(10)
]

pd.DataFrame(
    data=[
        [row[0][0], row[0][1], row[0][2], row[0][3], row[1]] for row in contributions_4d
    ]
).to_csv("data/distances/4d_1.csv", index=False, header=False, sep=" ")

new_contributions_4d = contributions_4d + [((50, 50, 50, 50), -1)]

pd.DataFrame(
    data=[
        [row[0][0], row[0][1], row[0][2], row[0][3], row[1]]
        for row in new_contributions_4d
    ]
).to_csv("data/distances/4d_2.csv", index=False, header=False, sep=" ")

In [11]:
assert (
    difference_ECP(
        contributions_4d, new_contributions_4d, (0, 100, 0, 100, 0, 100, 0, 100)
    )
    == 50**4
)